In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import re


In [27]:
driver =webdriver.Firefox()

In [29]:
#getting a professor
# driver.get("https://www.ratemyprofessors.com/professor/54889") #16 reviews
# driver.get("https://www.ratemyprofessors.com/professor/2633823") #zero reviews
# driver.get("https://www.ratemyprofessors.com/professor/131047") #54 reviews
driver.get("https://www.ratemyprofessors.com/professor/2584105") #empty

In [30]:
#closes the popup for cookies
try:
    driver.find_element(By.XPATH, "//div[starts-with(@class, 'FullPageModal__')]//button").click()
except NoSuchElementException:
    print('no popup found')

In [31]:
#get number of reviews and load all
review_count_finder = driver.find_element(By.XPATH, "//div[starts-with(@class, 'RatingValue__NumRatings')]").text
#find the number of ratings
review_count = 0
number_checker = re.findall(r'\b\d+\b', review_count_finder)
if number_checker:
    review_count = int(number_checker[0])

print(review_count)

#scroll down if more that 20 reviews
scroll_count = int(np.ceil((review_count-20)/10))

if scroll_count >= 1:
    load_more_button = driver.find_element(By.XPATH, "//button[starts-with(@class, 'Buttons__Button-sc')]")
    wait = WebDriverWait(driver, timeout = 3)

    for i in range(scroll_count):
        y_scroll_offset = 80
        wait.until(lambda d : load_more_button.is_displayed())
        button_y_position = load_more_button.rect['y']-y_scroll_offset
        driver.execute_script(f"window.scrollTo(0, {button_y_position})")
        load_more_button.click()


0


In [32]:
#get classes
class_list = []
class_cards = driver.find_elements(By.XPATH, "//div[starts-with(@class, 'RatingHeader__StyledClass')]")
for i in class_cards:
    class_list.append(i.text)
print(class_list)

[]


In [33]:
#get review date
date_list = []
date_cards = driver.find_elements(By.XPATH, "//div[starts-with(@class, 'TimeStamp__StyledTimeStamp')]")
for i in date_cards:
    date_list.append(i.text)
print(date_list)

[]


In [10]:
#get textbook 
textbook_list = []
textbook_cards = driver.find_elements(By.XPATH, "//div[starts-with(@class, 'MetaItem__StyledMetaItem')]")
for i in textbook_cards:
    textbook_list.append(i.text)
print(textbook_list)

['Textbook: Yes', 'Textbook: Yes', 'Textbook: No']


In [34]:
#get professor name 
professor_name = driver.find_element(By.XPATH, "//div[starts-with(@class, 'NameTitle__Name')]").text.strip().split()
professor_firstname = professor_name[0]
professor_lastname = professor_name[1]

#get all reviews and all associated elements!
review_cards = driver.find_elements(By.XPATH, "//div[starts-with(@class, 'Rating__RatingBody')]")
# print(len(review_cards))
# print(review_cards)
print(f'There are {len(review_cards)} reviews')
class_list = []
quality_list = []
difficulty_list = []
time_list = []
textbook_list = []
takeagain_list = []
comment_list = []
professor_firstname_list = []
professor_lastname_list = []
tags_list = []
attendance_list = []


for i in range(len(review_cards)):
    review_card = review_cards[i]
    professor_firstname_list.append(professor_firstname)
    professor_lastname_list.append(professor_lastname)
    class_list.append(review_card.find_element(By.XPATH, "(.//div[starts-with(@class, 'RatingHeader__StyledClass')])[2]").text)
    time_list.append(review_card.find_element(By.XPATH, "(.//div[starts-with(@class, 'TimeStamp__StyledTimeStamp')])[2]").text)
    quality_list.append(review_card.find_element(By.XPATH, "(.//div[starts-with(@class, 'CardNumRating')])[3]").text)
    difficulty_list.append(review_card.find_element(By.XPATH, "(.//div[starts-with(@class, 'CardNumRating')])[6]").text)
    comment_list.append(review_card.find_element(By.XPATH, "(.//div[starts-with(@class, 'Comments__StyledComments')])[1]").text)
    #
    try:
        individual_meta_lists = (review_card.find_elements(By.XPATH, "(.//div[starts-with(@class, 'MetaItem__StyledMetaItem')])"))
        takeagain_flag = False
        textbook_flag = False
        attendance_flag = False
        for meta_item in individual_meta_lists:
            #scraping for takeagain list in meta items
            if "Would Take Again:" in meta_item.text:
                takeagain_list.append(meta_item.text.replace('Would Take Again:','').strip())
                takeagain_flag = True
            #scraping for textbook requirement in meta items
            if "Textbook:" in meta_item.text:
                textbook_list.append(meta_item.text.replace('Textbook:','').strip())
                textbook_flag = True
            #scraping for attendance requirement in meta items
            if "Attendance:" in meta_item.text:
                attendance_list.append(meta_item.text.replace('Attendance:','').strip())
                attendance_flag = True
        if takeagain_flag == False:
            takeagain_list.append('unknown')
        if textbook_flag == False:
            textbook_list.append('unknown')
        if attendance_flag == False:
            attendance_list.append('unknown')
    except NoSuchElementException:
        print('meta tag error')
    #getting the tags in each review
    try:
        individual_tag_list = review_card.find_elements(By.XPATH, "(.//span[starts-with(@class, 'Tag-bs9vf4')])")
        temp_tag_list = []
        for tag in individual_tag_list:
            temp_tag_list.append(tag.text.strip())
        tags_list.append(temp_tag_list)
    except NoSuchElementException:
        tags_list.append([])

print(professor_firstname_list)
print(professor_lastname_list)
print(class_list)
print(time_list)
print(quality_list)
print(difficulty_list)
print(textbook_list)
print(takeagain_list)
print(attendance_list)
print(comment_list)
print(tags_list)

There are 0 reviews
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [92]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia_list = []
for comment in comment_list:
    sia_list.append(sia.polarity_scores(comment)['compound'])

print(sia_list)



[0.6597, 0.4576, 0.4528, 0.8646, 0.9634, 0.9636, 0.7482, 0.7892, 0.8169, 0.1706, 0.8906, 0.0, 0.2255, 0.9113, 0.9299, 0.5267, 0.9638, 0.5918, 0.9123, 0.3612, -0.4878, 0.8918, 0.4927, -0.3853, 0.669, 0.7845, 0.938, 0.5975, 0.4404, 0.8481, 0.4927, 0.6369, 0.541, -0.4495, 0.4203, 0.4391, 0.961, 0.1513, 0.5719, 0.7717, 0.7346, 0.765, 0.8555, 0.8834, 0.8119, 0.7783, 0.4404, 0.6588, -0.296, 0.7951, -0.296, 0.6369, 0.802, 0.6705]


In [93]:
#must be done as a prepatory step
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/joshch/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [105]:
from tqdm import tqdm
import time

progressbar = tqdm (range (len(review_cards)), desc="Loading…", ascii=False, ncols=75)

for i in progressbar:
    review_card = review_cards[i]
    professor_firstname_list.append(professor_firstname)
    professor_lastname_list.append(professor_lastname)
    class_list.append(review_card.find_element(By.XPATH, "(.//div[starts-with(@class, 'RatingHeader__StyledClass')])[2]").text)
    time_list.append(review_card.find_element(By.XPATH, "(.//div[starts-with(@class, 'TimeStamp__StyledTimeStamp')])[2]").text)
    quality_list.append(review_card.find_element(By.XPATH, "(.//div[starts-with(@class, 'CardNumRating')])[3]").text)
    difficulty_list.append(review_card.find_element(By.XPATH, "(.//div[starts-with(@class, 'CardNumRating')])[6]").text)
    comment_list.append(review_card.find_element(By.XPATH, "(.//div[starts-with(@class, 'Comments__StyledComments')])[1]").text)
    #
    try:
        individual_meta_lists = (review_card.find_elements(By.XPATH, "(.//div[starts-with(@class, 'MetaItem__StyledMetaItem')])"))
        takeagain_flag = False
        textbook_flag = False
        attendance_flag = False
        for meta_item in individual_meta_lists:
            #scraping for takeagain list in meta items
            if "Would Take Again:" in meta_item.text:
                takeagain_list.append(meta_item.text.replace('Would Take Again:','').strip())
                takeagain_flag = True
            #scraping for textbook requirement in meta items
            if "Textbook:" in meta_item.text:
                textbook_list.append(meta_item.text.replace('Textbook:','').strip())
                textbook_flag = True
            #scraping for attendance requirement in meta items
            if "Attendance:" in meta_item.text:
                attendance_list.append(meta_item.text.replace('Attendance:','').strip())
                attendance_flag = True
        if takeagain_flag == False:
            takeagain_list.append('unknown')
        if textbook_flag == False:
            textbook_list.append('unknown')
        if attendance_flag == False:
            attendance_list.append('unknown')
    except NoSuchElementException:
        print('meta tag error')
    #getting the tags in each review
    try:
        individual_tag_list = review_card.find_elements(By.XPATH, "(.//span[starts-with(@class, 'Tag-bs9vf4')])")
        temp_tag_list = []
        for tag in individual_tag_list:
            temp_tag_list.append(tag.text.strip())
        tags_list.append(temp_tag_list)
    except NoSuchElementException:
        tags_list.append([])

print(professor_firstname_list)
print(professor_lastname_list)
print(class_list)
print(time_list)
print(quality_list)
print(difficulty_list)
print(textbook_list)
print(takeagain_list)
print(attendance_list)
print(comment_list)
print(tags_list)

Loading…: 100%|████████████████████████████| 54/54 [00:02<00:00, 18.71it/s]

['Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan', 'Alan',

In [38]:
import pickle
file_list = ['Chris_Stone_data.pkl', 'Francis_Su_data.pkl', 'Joseph_King_data.pkl', 'Ziyad_Duron_data.pkl']
for file in file_list:
    with open(file, 'rb') as fp:
        person = pickle.load(fp)
        print('Person dictionary')
        print(person)

FileNotFoundError: [Errno 2] No such file or directory: 'Chris_Stone_data.pkl'

In [25]:
file = 'professor_urls.pkl'
with open(file, 'rb') as fp:
    professor_urls = pickle.load(fp)
    print(professor_urls)
    print(len(professor_urls))


['https://www.ratemyprofessors.com/professor/2495362', 'https://www.ratemyprofessors.com/professor/2553953', 'https://www.ratemyprofessors.com/professor/2584104', 'https://www.ratemyprofessors.com/professor/2584105']
4


In [37]:
#writing into pickle file
import pickle
import os
script_dir = os.path.dirname(__file__)
relative_directory = 'tester1'
relative_file = 'testerwrite.pkl'
file = os.path.join(script_dir, relative_directory, relative_file)
file_list = [professor_urls[0], professor_urls[1], professor_urls[2], professor_urls[3]]

with open(file, 'wb') as fp:
    pickle.dump(file_list, fp)

NameError: name '__file__' is not defined